<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Start-up" data-toc-modified-id="Start-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Start up</a></span></li></ul></div>

<img align="left" src="images/P005381-obverse-photo.png" width="15%"/>
<img align="left" src="images/P005381-obverse-lineart-annot.png" width="15%"/>
<img align="right" src="images/P005381-reverse-photo.png" width="15%"/>
<img align="right" src="images/P005381-reverse-lineart.png" width="15%"/>

<p>
```
&P005381 = MSVO 3, 70
```
</p>
<p>
<img src="images/P005381-obverse-atf.png" width="40%"/>
<img src="images/P005381-reverse-atf.png" width="40%"/>
</p>

<img align="right" src="images/tf-small.png"/>


# Numbers

## Start up

We import the Python modules we need.

In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import os, sys, collections
from IPython.display import display

In [2]:
REPO = f'~/github/Dans-labs/Nino-cunei'
PROGRAM_DIR = os.path.expanduser(f'{REPO}/programs')
sys.path.append(PROGRAM_DIR)
from cunei import Cunei
CN = Cunei(REPO)
CN.api.makeAvailableIn(globals())

Found 2724 tablet linearts
Found 2095 ideographs



**Documentation:**
[Feature docs](https://github.com/Dans-labs/Nino-cunei/blob/master/docs/transcription.md)
[Cunei API](https://github.com/Dans-labs/Nino-cunei/blob/master/docs/cunei.md)
[Text-Fabric API](https://github.com/Dans-labs/text-fabric)


In [3]:
def dm(markdown): display(Markdown(markdown))

Specification of the Shin systems: just the bare minimum of info.

In [4]:
numberSystems = dict(
    shinP = (40, 3, 18, 24, 45),
    shinPP = (4,19, 36, 41, 46, 49),
    shinS = (25, 27, 28, 42, 5, 20, 47, 37),
)

We turn the numbers into numeral graphemes:

In [5]:
systems = {}

for (shin, numbers) in numberSystems.items():
    systems[shin] = {f'N{n:>02}' for n in numbers}

Reality check

In [6]:
systems

{'shinP': {'N03', 'N18', 'N24', 'N40', 'N45'},
 'shinPP': {'N04', 'N19', 'N36', 'N41', 'N46', 'N49'},
 'shinS': {'N05', 'N20', 'N25', 'N27', 'N28', 'N37', 'N42', 'N47'}}

We also want the opposite: given a numeral, which system is it?

In [7]:
numeralMap = {}

for (shin, numerals) in systems.items():
    for n in numerals:
        if n in numeralMap:
            dm(f'**warning:** Numeral {n} in {shin} was already in {numeralMap[n]}')
        numeralMap[n] = shin

numeralMap

{'N03': 'shinP',
 'N04': 'shinPP',
 'N05': 'shinS',
 'N18': 'shinP',
 'N19': 'shinPP',
 'N20': 'shinS',
 'N24': 'shinP',
 'N25': 'shinS',
 'N27': 'shinS',
 'N28': 'shinS',
 'N36': 'shinPP',
 'N37': 'shinS',
 'N40': 'shinP',
 'N41': 'shinPP',
 'N42': 'shinS',
 'N45': 'shinP',
 'N46': 'shinPP',
 'N47': 'shinS',
 'N49': 'shinPP'}

Exercise:

For each tablet, add three properties: hasShinP, hasShinPP, hasShinS.
They will be True if and only if the tablet has a numeral in that category.
Even better, instead of True or False, we let them record how many numerals in that set they have. 

In [10]:
tabletNumerics = collections.defaultdict(collections.Counter)

for tablet in F.otype.s('tablet'):
    pNum = F.catalogId.v(tablet)
    for sign in L.d(tablet, otype='sign'):
        if F.type.v(sign) == 'numeral':
            numeral = F.grapheme.v(sign)
            system = numeralMap.get(numeral, None)
            if system is not None:
                tabletNumerics[pNum][system] += 1

Now we write a csv file to the report directory, so that you can work with the data in Excel.

We show the first few lines in the notebook

In [12]:
filePath = f'{CN.reportDir}/tabletNumerics.tsv'
lines = []
systemNames = sorted(systems)
fieldNames = "\t".join(systemNames)
for pNum in sorted(tabletNumerics):
    data = tabletNumerics[pNum]
    values = "\t".join(str(data[s]) for s in systemNames)
    lines.append(f'{pNum}\t{values}\n')
with open(filePath, 'w') as fh:
    fh.write(f'tablet\t{fieldNames}\n')
    fh.write(''.join(lines))

print(''.join(lines[0:10]))

P000148	0	0	1
P000245	2	0	0
P000266	0	1	0
P000308	2	0	0
P000434	2	0	0
P000511	1	0	0
P000517	1	0	0
P000550	2	0	0
P000734	2	0	0
P000735	2	0	1

